# VARS-TOOL Quick Start Tutorial

## Install

This is a simple tutorial show casing VARS-TOOL functionalities that are being implement in Python.Let's install the latest VARS-TOOL from GitHub:

In [1]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...


Processing c:\users\corde\giws\vars-tool\tutorial\vars-tool
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=427236 sha256=6f4e6dc428c9bc380c876a40f9db397c06c0cd1c2d8abc73db4580e1a62e3da2
  Stored in directory: c:\users\corde\appdata\local\pip\cache\wheels\ea\22\e6\94dfc4ddef885e03d97899daed3e63a6c2c3a1295f1b37f687
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## Example using the Ishigami and wavy6D models

Let's import VARS for the analysis and the Model class for creating a wrapper around the desired model so that it can be inputted into VARS

In [2]:
# importing VARS and Model from varstool package
from varstool import VARS, Model

# these are being imported in order for us to display the data nicely on the notebook
import numpy as np
import pandas as pd

Here is a quick function definition of the Ishigami model:

In [3]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

Here is a quick function definition of the wavy6D model:

In [4]:
def wavy6D(x):
    '''wavy6D test function'''
    
# The features include:             
# (1) The 6 input factors are non-interacting                  
# (2) The function is "wavy" along the first 5 dimensions but with different 
#     frequencies and amplitude at different directions                                     
# (3) The last dimension (X6) is fully isensitive (dummy variable)                                         
# (4) It possess large-scale features such multi-modality (X1) 
#     to small-scale features such as roughness (X3)          
# (5) The directional variograms of X1 and X2 cross each other
    
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 6:
        raise ValueError('`x` must have only five arguments at a time')
    
    y1 = -1*np.sin(2*np.pi*x[0]/2) - 0.3*np.sin(2*np.pi*x[0]/0.6)
    y2 = 0.76*(-1*np.sin(2*np.pi*(x[1]-0.2)/2)) - 0.315
    y3 = 0.12*(-1*np.sin(2*np.pi*(x[2]-0.2)/1.9)) + 0.02*(-1*np.sin(2*np.pi*x[2]/0.021)) - 0.96
    y4 = 0.12*(-1*np.sin(2*np.pi*(x[3]-0.2)/1.9))-0.97
    y5 = 0.05*(-1*np.sin(2*np.pi*(x[4]-0.2)/2))-1.02
    y6 = -1.08

    return y1 + y2 + y3 + y4 + y5 + y6

As mentioned previously The `Model` class is a wrapper for custom functions(models) in the online version of VARS. However, the first paramter of every function must accept an array of parameters

Here we will create two wrappers for the testing models ishigami and wavy6D:

In [5]:
ishigami_model = Model(ishigami)

In [6]:
wavy6D_model = Model(wavy6D)

When creating an experiment you will need to assign a variable to an instance of VARS as shown below.

The paramters of the VARS class are described as:

**paramaters**: the name of each paramter along with their upper and lower bounds

**num_stars**: the total number of star samples that are desired for STAR-VARS analysis

**delta_h**: the sampling resolution of the VARS analysis

**ivars_scales**: the scales that are to be used when doing IVARS, e.g, 0.1 and 0.3 correspond (0-0.1) and (0-0.3) <br /> note: can not have a scale larger then 0.5

**star_centres**: This is only used if a sampler is not chosen and you are wanting to generate your own star centres(randomized numbers)

**sampler**: the sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers.

**seed**: the seed number for randomization of the sampling strategy specified by `sampler`, only needed if a sampler was chosen

**model**: the wrapper of your model you made when using the `Model` class

**bootstrap_flag**: this is a True/False value that specifies if bootstrapping will be used in the VARS analysis

**bootstrap_size**: the number of sampling iterations with replacement

**bootstrap_ci**: the bootstrap-based confidence interval on results

**grouping_flag**: this is a True/False value that specifies if paramater grouping will be used in the VARS analysis

**num_grps**: the number of groups you want to split your paramaters into, if left blank the optimal number of groups will be calculated by VARS

Create `experiment_1` and `experiment_2` then initialize the values needed to run a VARS analysis:

`experiment_1` is an instance of VARS that is using the ishigami model

In [9]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

`experiment_2` is an instance of VARS that is using the wavy6D model

In [8]:
experiment_2 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1), \
                                  'x4':(0, 1), 'x5':(0, 1), 'x6': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = wavy6D_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

A report displaying the current status of the VARS analysis can be found by typing in the variable name of the instance you created, here this is `experiment_1`

In [9]:
experiment_1

Star Centres: Loaded
Star Points: Not Loaded
Parameters: 6 paremeters set
Delta h: 0.1
Model: wavy6D
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Not Done

To run the VARS analysis we can simply do the following:

note: we are using `experiment_1` here but `experiment_2` can be used in the same exact way

In [10]:
experiment_1.run_online()

Now if we take a look at the status report again we can see that the `Vars Analysis` is now done.

In [11]:
experiment_1

Star Centres: Loaded
Star Points: Loaded
Parameters: 3 paremeters set
Delta h: 0.1
Model: ishigami
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Done

Now we can access all the results using 'dot' notation on our created instance:

**Directional variogram results**

There are 10 rows as the number of stars was 10, and each at a resolution incremented by 0.1 which was specificed to be the sampling resolution

In [12]:
experiment_1.variogram_value.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.003726,0.146650,0.000007
0.2,0.015010,0.595799,0.000022
0.3,0.033920,1.349779,0.000040
0.4,0.060397,2.394087,0.000059
0.5,0.094257,3.696488,0.000079
0.6,0.135186,5.207548,0.000102
0.7,0.182747,6.862652,0.000135
0.8,0.236382,8.585457,0.000185
0.9,0.295426,10.292503,0.000264


**Integrated variogram**

In [13]:
experiment_1.ivars

,x1,x2,x3
0.1,0.000186,0.007332,3.561100e-07
0.3,0.003570,0.141734,4.950164e-06
0.5,0.016018,0.633456,1.674673e-05


**VARS-TO: Sobol total-order effects calculated through VARS**

In [14]:
experiment_1.sobol_value

param
x1    0.019410
x2    0.751635
x3    0.000016
dtype: float64

**VARS-ABE: Morris mean absolute elementary effects across scales**

In [15]:
experiment_1.morris_value[0].unstack(0)

param,x1,x2,x3
h,,,
0.1,0.085391,0.503689,0.002496
0.2,0.171796,1.032671,0.004589
0.3,0.258785,1.576551,0.006452
0.4,0.345921,2.124345,0.008257
0.5,0.432766,2.664763,0.010177
0.6,0.518882,3.186517,0.012385
0.7,0.603831,3.678620,0.015054
0.8,0.687183,4.130684,0.018355
0.9,0.768518,4.533205,0.022461


**VARS-ACE: Morris mean actual elementary effects across scales**

In [16]:
experiment_1.morris_value[1].unstack(0)

param,x1,x2,x3
h,,,
0.1,0.085391,0.503689,0.002496
0.2,0.171796,1.032671,0.004589
0.3,0.258785,1.576551,0.006452
0.4,0.345921,2.124345,0.008257
0.5,0.432766,2.664763,0.010177
0.6,0.518882,3.186517,0.012385
0.7,0.603831,3.678620,0.015054
0.8,0.687183,4.130684,0.018355
0.9,0.768518,4.533205,0.022461


**The factor(parameter) rankings based on their influence**. 

The influence is based on how large or small a result is. The lower the ranking the more influential(larger) a factor is.

In [17]:
experiment_1.ivars_factor_ranking

,x1,x2,x3
0.1,1,0,2
0.3,1,0,2
0.5,1,0,2


In [18]:
experiment_1.sobol_factor_ranking

,x1,x2,x3
0,1,0,2


**Bootstrapping results based on the confidence interval limits**

low is the lower limit of the results after `bootstrap_size` sampling iterations

upp is the upper limit of the results after `bootstrap_size` sampling iterations

In [19]:
experiment_1.variogram_low

h,x1,x2,x3
0.1,0.003659,0.143081,0.000005
0.2,0.014739,0.579480,0.000017
0.3,0.033305,1.309094,0.000030
0.4,0.059290,2.316132,0.000044
0.5,0.092512,3.568480,0.000059
0.6,0.132663,5.018336,0.000077
0.7,0.179315,6.604163,0.000102
0.8,0.231923,8.253882,0.000139
0.9,0.289839,9.889098,0.000198


In [20]:
experiment_1.variogram_upp

h,x1,x2,x3
0.1,0.003809,0.150279,0.000009
0.2,0.015346,0.612394,0.000027
0.3,0.034683,1.391149,0.000048
0.4,0.061763,2.473352,0.000069
0.5,0.096398,3.826649,0.000092
0.6,0.138267,5.399942,0.000120
0.7,0.186922,7.125490,0.000159
0.8,0.241794,8.922610,0.000217
0.9,0.302198,10.702696,0.000311


In [21]:
experiment_1.sobol_low

param,x1,x2,x3
,0.016749,0.656539,0.000011


In [22]:
experiment_1.sobol_upp

param,x1,x2,x3
,0.027285,1.061238,0.000023


In [23]:
experiment_1.ivars_low

,x1,x2,x3
0.1,0.000183,0.007154,2.668938e-07
0.3,0.003505,0.137711,3.714735e-06
0.5,0.015725,0.613203,1.258522e-05


In [24]:
experiment_1.ivars_upp

,x1,x2,x3
0.1,0.00019,0.007514,4.255673e-07
0.3,0.00365,0.145825,5.884990e-06
0.5,0.01638,0.654050,1.981167e-05


**Reliability estimates of factor rankings based on VARS-TO**

reliability estimates give the ratio of how many bootstrapped rankings were the same as the initial rankings for `bootstrap_size` sampling iterations

In [25]:
experiment_1.rel_sobol_factor_ranking

,x1,x2,x3
0,1.0,1.0,1.0


In [26]:
experiment_1.rel_ivars_factor_ranking

,x1,x2,x3
0.1,1.0,1.0,1.0
0.3,1.0,1.0,1.0
0.5,1.0,1.0,1.0


**Factor grouping based on IVARS50 and Sobol results**

In [27]:
experiment_1.ivars50_grp

,x1,x2,x3
0,1,1,2


In [28]:
experiment_1.sobol_grp

,x1,x2,x3
0,1,1,2


**Reliability estimates of rankings based on grouping**

In [29]:
experiment_1.reli_sobol_grp

,x1,x2,x3
0,1.0,1.0,1.0


In [30]:
experiment_1.reli_ivars50_grp

,x1,x2,x3
0,1.0,1.0,1.0


**Directional covariogram results**

In [31]:
experiment_1.covariogram_value.unstack(0)

param,x1,x2,x3
h,,,
0.1,2.821517,1.926224,2.789734
0.2,2.805044,1.290550,2.789599
0.3,2.786017,0.521457,2.789589
0.4,2.764611,-0.351030,2.789697
0.5,2.741037,-1.291146,2.789923
0.6,2.715537,-2.258778,2.790268
0.7,2.688383,-3.211066,2.790737
0.8,2.659872,-4.104146,2.791336
0.9,2.630327,-4.894949,2.792078


**Directional expected covariogram results**

In [32]:
experiment_1.e_covariogram_value.unstack(0)

param,x1,x2,x3
h,,,
0.1,0.048258,1.866380,0.000035
0.2,0.031985,1.230827,0.000018
0.3,0.012959,0.461739,0.000003
0.4,-0.008646,-0.410867,-0.000011
0.5,-0.032619,-1.351224,-0.000024
0.6,-0.058719,-2.319215,-0.000038
0.7,-0.086670,-3.271979,-0.000054
0.8,-0.116172,-4.165642,-0.000073
0.9,-0.146898,-4.957124,-0.000096
